# Customer Segmentation - 고객 군집

In [1]:
import warnings
warnings.filterwarnings('ignore')

# I. Import Packages and Google Drive Mount

> ## 1) Import Packages

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

> ## 2) Google Drive Mount
* 'Online_Retail.zip' 파일을 구글드라이브에 업로드 후 진행

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


* 마운트 결과 확인

In [4]:
!ls -l '/content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip'

ls: cannot access '/content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip': No such file or directory


# II. Data Preprocessing

> ## 1) Unzip 'Online_Retail.zip'

* Colab 파일시스템에 'Online_Retail.csv' 파일 생성

In [5]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/datasets/Online_Retail.zip

unzip:  cannot find or open /content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip, /content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip.zip or /content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip.ZIP.


* Online_Retail.zip 파일 확인

In [6]:
!ls -l

total 8
drwx------ 6 root root 4096 Jul 19 02:12 drive
drwxr-xr-x 1 root root 4096 Jul 15 13:38 sample_data


> ## 2) 데이터 읽어오기

* pandas DataFrame

In [7]:
%%time

DF = pd.read_excel('Online_Retail.xlsx')

DF.info()

FileNotFoundError: ignored

> ## 3) 데이터 설명

* InvoiceNo : 주문 번호, 'C' 시작은 주문취소
* StockCode : 제품 코드(Item Code)
* Description : 제품 설명
* Quantity : 주문 건수
* InvoiceDate : 주문 날짜
* UnitPrice : 제품 단가
* CustomerID : 고객번호
* Country : 국가명(주문 고객 국적)

In [ ]:
DF.head(3)

> ## 4) 결측치 제거

* 'Quantity', 'UnitPrice', 'CustomerID'

In [ ]:
DF = DF[DF['Quantity'] > 0]
DF = DF[DF['UnitPrice'] > 0]
DF = DF[DF['CustomerID'].notnull()]

DF.shape

* 결과 확인

In [ ]:
DF.isnull().sum(axis = 0)

> ## 5) 'United Kingdom' 만 사용

* 대부분의 구매자가 영국국적

In [ ]:
DF['Country'].value_counts()[:10]

* 영국 데이터만 추출

In [ ]:
DF = DF[DF['Country'] == 'United Kingdom']

DF.shape

# III. RFM 기반 Data Reengineering

* RECENCY : 가장 최근 상품 구매일에서 오늘까지 기간
* FREQUENCY : 상품 구매 횟수
* MONETARY VALUE : 총 구매 금액

> ## 1) 'sale_amount' 추가

* '주문 금액' = '주문 개수' * '제품 가격'

In [ ]:
DF['sale_amount'] = DF['Quantity'] * DF['UnitPrice']

In [ ]:
DF.head()

* 'Customer ID' 정수 변환

In [ ]:
DF['CustomerID'] = DF['CustomerID'].astype(int)

> ## 2) 주문 특성 확인

* Top-5 주문 건수 'Customer ID'

In [ ]:
DF['CustomerID'].value_counts().head(5)

* Top-5 주문 금액 'Customer ID'

In [ ]:
DF.groupby('CustomerID')['sale_amount'].sum().sort_values(ascending = False)[:5]

* 몇몇 특정 고객이 많은 '주문 건수'와 '주문 금액'을 가지고 있음

> ## 3) 'Customer ID' 기준으로 DataFrame 재구성

* Recency : 'InvoiceDate'의 max()
* Frequency : 'InvoiceNo'의 count()
* Monetary value : 'sale_amount'의 sum()

In [ ]:
aggregations = {'InvoiceDate': 'max',
                'InvoiceNo': 'count',
                'sale_amount':'sum'}

DF_CID = DF.groupby('CustomerID').agg(aggregations)

* groupby( ) 결과 Index와 Column 이름 수정
 - 'Recency', 'Frequency', 'Monetary'

In [ ]:
DF_CID = DF_CID.rename(columns = {'InvoiceDate':'Recency',
                                  'InvoiceNo':'Frequency',
                                  'sale_amount':'Monetary'
                                   })

DF_CID = DF_CID.reset_index()

* RMF 변경 결과

In [ ]:
DF_CID.head()

> ## 4) 최근 구매일자 Day로 변경

* 오늘 날짜에서 최근 구매날짜('Recency')를 뺀 일자

In [ ]:
import datetime as dt

DF_CID['Recency'] = dt.datetime(2011, 12, 10) - DF_CID['Recency']
DF_CID['Recency'] = DF_CID['Recency'].apply(lambda x: x.days + 1)

* 처리 결과 확인

In [ ]:
DF_CID.head()

> ## 5) Visualization

* 변수 별 분포 확인

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize = (12, 4), nrows = 1, ncols = 3)

ax1.set_title('Recency Histogram')
ax1.hist(DF_CID['Recency'])

ax2.set_title('Frequency Histogram')
ax2.hist(DF_CID['Frequency'])

ax3.set_title('Monetary Histogram')
ax3.hist(DF_CID['Monetary'])

plt.show()

# IV. K-means Clustering

> ## 1) Standardization

* 표준화를 통한 평균과 표준편차 Scaling

In [ ]:
DF_CID[['Recency', 'Frequency', 'Monetary']].describe()

* Scaling 수행

In [ ]:
from sklearn.preprocessing import StandardScaler

X_input = DF_CID[['Recency','Frequency','Monetary']].values
X_input_scaled = StandardScaler().fit_transform(X_input)

> ## 2) Modeling

* K-means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 5,
                random_state = 2045)

kmeans.fit_predict(X_input_scaled)

> ## 3) Silhouette Score

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_score(X_input_scaled, kmeans.fit_predict(X_input_scaled))

# 
# 
# 
# The End
# 
# 
# 